In [ ]:
!pip install ta

In [1]:
import sys
import os

# Add the root directory "chaoticX" to the system path
sys.path.append(os.path.abspath(".."))  # one level up from Notebooks/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
from Data.timeFrames import timeFrame
from Data.binanceAPI import BinanceAPI
from Core.zone_detection import ZoneDetector
from Core.zone_merge import ZoneMerger
from Core.zone_reactions import ZoneReactor
from ML.Model import ModelHandler


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [3]:
df = pd.read_csv('../Datasets/train_dataset.csv')


In [4]:
modelHandler = ModelHandler(model_type='xgb')
modelHandler.load('../Models/model.pkl')
data = modelHandler.getFeatureImportance(feature_names=list(df.columns))
sorted_importance = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))

sorted_importance

{'touch_type': 15.917057991027832,
 'distance_to_nearest_zone_above': 13.719236373901367,
 'count': 9.527777671813965,
 'conf_1h_count': 7.502933979034424,
 'below_ema 20': 7.195952892303467,
 'above_conf_count_BuLiq': 5.7219085693359375,
 'below_type': 5.520761966705322,
 'distance_to_nearest_zone_below': 5.377983093261719,
 'above_atr': 5.340757846832275,
 'conf_count_BuFVG': 5.294818878173828,
 'candle_high': 4.895845890045166,
 'below_zone_low': 4.634347915649414,
 'below_conf_count_BuLiq': 4.224490165710449,
 'volume_on_creation': 4.195095539093018,
 'above_conf_count_BrFVG': 4.169815540313721,
 'below_avg_volume_past_5': 4.129554748535156,
 'candle_open': 4.053562164306641,
 'candle_close': 3.984543800354004,
 'conf_count_BuLiq': 3.963528633117676,
 'conf_15min_count': 3.5624382495880127,
 'below_zone_width': 3.4329915046691895,
 'below_avg_volume_around_zone': 3.4091732501983643,
 'avg_volume_past_5': 3.3649494647979736,
 'conf_count_BrOB': 3.323895215988159,
 'below_rsi': 3.248

In [7]:
below_features = {key : value for key, value in sorted_importance.items() if key.startswith('below_') and value < 2}
below_features

{'below_ema 50': 1.9022575616836548,
 'below_body_size': 1.7191979885101318,
 'below_avg_ema_20': 1.6693785190582275,
 'below_conf_count_BuOB': 1.5509389638900757,
 'below_conf_count_BuFVG': 1.500038504600525,
 'below_conf_count_BrLiq': 1.4552593231201172,
 'below_conf_count_BrFVG': 0.9100236892700195}

In [8]:
above_features = {key : value for key, value in sorted_importance.items() if key.startswith('above_') and value < 2}
above_features

{'above_conf_1h_count': 1.8370522260665894,
 'above_conf_count_BrLiq': 1.490531086921692,
 'above_conf_count_BrOB': 1.1504571437835693,
 'above_time_frame': 0.7534046769142151,
 'above_conf_1D_count': 0.48588109016418457,
 'above_avg_rsi': 0.3922140598297119}

In [12]:
correlations = df.corr()['target'].sort_values(ascending=False)
correlations_dict = correlations.to_dict()
{key : value for key,value in correlations_dict.items() if  pd.notna(value) or value > 0.1 or value < -0.1}


{'target': 1.0,
 'above_time_frame': 0.3679320746300365,
 'above_conf_count_BuLiq': 0.32349983828793344,
 'above_wick_ratio': 0.2788409060828912,
 'above_conf_is_buy_zone': 0.27803835422704554,
 'distance_to_nearest_zone_above': 0.27750098171455184,
 'above_conf_count_BrLiq': 0.2729358750076576,
 'distance_to_nearest_zone_below': 0.2278224402901493,
 'above_zone_width': 0.2179695235238414,
 'above_conf_1h_count': 0.20407302013541723,
 'above_atr_mean': 0.1989489561604861,
 'above_body_size': 0.19493945503478988,
 'above_conf_count_BuFVG': 0.18134969920707475,
 'momentum_5': 0.17340371076670666,
 'above_conf_count_BuOB': 0.17224058388735491,
 'touch_type': 0.17074302384753248,
 'rsi': 0.16675392036937475,
 'above_conf_count_BrFVG': 0.16339976973597548,
 'below_type': 0.15351445124399332,
 'below_liquidity_height': 0.14885594802511956,
 'below_avg_swing_strength': 0.1488558235669987,
 'above_conf_count_BrOB': 0.14322267617459053,
 'below_avg_rsi': 0.14222243323783554,
 'below_equal_level

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/OHLC.csv')

In [ ]:
df.loc[~((df['is_target'] == 1) | (df['is_target'] == 0))]


In [ ]:
for col in df.columns:
    print(f'{col} : {df[col].dtype}')

In [ ]:
df.shape

In [ ]:
api = BinanceAPI()
df_4h = api.get_ohlcv(interval='4h')
df_1d = api.get_ohlcv(interval='1D')
df_1h = api.get_ohlcv()

In [ ]:
detector1h = ZoneDetector(df_1h)
zone_1h = detector1h.get_zones()
detector4h = ZoneDetector(df_4h,timeframe= "4h")
zone_4h = detector4h.get_zones()
detector1d = ZoneDetector(df_1d,timeframe="1D")
zone_1d = detector1d.get_zones()


In [ ]:
merger = ZoneMerger(zone_1h+zone_4h+zone_1d)

zones = merger.merge()
#zones = merger.add_liq_confluence(zones)

In [ ]:
reactor = ZoneReactor(df_1h, zones)
zones = reactor.get_zone_reaction()
zones = merger.getNearbyZone(zones)

In [ ]:
zones = reactor.get_next_target_zone()

In [ ]:
zones[-1]

In [ ]:
import requests

def get_orderbook_price(symbol, target_price):
    url = "https://api.binance.com/api/v3/depth"
    params = {"symbol": symbol.upper(), "limit": 10000}
    res = requests.get(url, params=params).json()

    target_price = str(float(target_price))  # Ensure formatting

    bids = {price: qty for price, qty in res["bids"]}
    asks = {price: qty for price, qty in res["asks"]}

    bid_qty = bids.get(target_price, None)
    ask_qty = asks.get(target_price, None)

    return {
        "bid_quantity": bid_qty,
        "ask_quantity": ask_qty,
        "response" : res
    }

# Example usage:
print(get_orderbook_price("BTCUSDT", 109000.00))


In [ ]:
df_zone_reaction['level'] = df_zone_reaction['level'].fillna((df_zone_reaction['zone_high']+df_zone_reaction['zone_low'])/2)
df_zone_reaction['zone_width'] = df_zone_reaction['zone_width'].fillna(0)
df_zone_reaction['body_size'] = df_zone_reaction['body_size'].fillna(0)
df_zone_reaction['wick_ratio'] = df_zone_reaction['wick_ratio'].fillna(0)
df_zone_reaction['volume_on_creation'] = df_zone_reaction['volume_on_creation'].fillna(0)
df_zone_reaction['avg_volume_past_5'] = df_zone_reaction['avg_volume_past_5'].fillna(0)
df_zone_reaction['prev_volatility_5'] = df_zone_reaction['prev_volatility_5'].fillna(0)
df_zone_reaction['momentum_5'] = df_zone_reaction['momentum_5'].fillna(0)
df_zone_reaction['avg_volume_around_zone'] = df_zone_reaction['avg_volume_around_zone'].fillna(0)
df_zone_reaction['equal_level_deviation'] = df_zone_reaction['equal_level_deviation'].fillna(0)
df_zone_reaction['duration_between_first_last_touch'] = df_zone_reaction['duration_between_first_last_touch'].fillna(0)

In [ ]:
df_zone_reaction['type'].unique()

In [ ]:
df_zone_reaction.to_csv('reactions.csv',index=False)

In [ ]:
df_zone_reaction.describe()

In [ ]:
df_zone_reaction.notna().sum()

In [ ]:
df_zone_reaction= df_zone_reaction.dropna()

In [ ]:
df_ob_fvg_reaction = df_zone_reaction.loc[~df_zone_reaction['type'].isin(['Buy-Side Liq','Sell-Side Liq'])]
df_liq_reaction = df_zone_reaction.loc[df_zone_reaction['type'] .isin(['Buy-Sidde Liq','Sell-Side Liq'])]

In [ ]:
reaction_type1 = list(df_ob_fvg_reaction.reaction_type.unique())
reaction_type2 = list(df_liq_reaction.reaction_type.unique())
zone_type = list(df_zone_reaction['type'].unique())

In [ ]:
df_ob_fvg_reaction['type'] = df_ob_fvg_reaction['type'].apply(lambda x : zone_type.index(x))
df_ob_fvg_reaction['nearest_zone_type'] = df_ob_fvg_reaction['nearest_zone_type'].apply(lambda x : zone_type.index(x))
df_liq_reaction['type'] = df_liq_reaction['type'].apply(lambda x : zone_type.index(x))
df_liq_reaction['nearest_zone_type'] = df_liq_reaction['nearest_zone_type'].apply(lambda x : zone_type.index(x))



In [ ]:
df_ob_fvg_reaction['reaction_type'] = df_ob_fvg_reaction['reaction_type'].apply(lambda x : reaction_type1.index(x))
df_liq_reaction['reaction_type'] = df_liq_reaction['reaction_type'].apply(lambda x : reaction_type2.index(x))

In [ ]:
df_zone_reaction.dtypes

In [ ]:
X1 = df_liq_reaction.drop(columns=['reaction_type','nearest_zone_index','index','zone_high','zone_low','duration_between_first_last_touch'])
Y1 = df_liq_reaction['reaction_type']
X2 = df_ob_fvg_reaction.drop(columns=['reaction_type','nearest_zone_index','index','zone_high','zone_low','duration_between_first_last_touch'])
Y2 = df_ob_fvg_reaction['reaction_type']

In [ ]:
from sklearn.model_selection import train_test_split

train_x1,val_x1,train_y1,val_y1 = train_test_split(X1,Y1,test_size=0.7)
train_x2,val_x2,train_y2,val_y2 = train_test_split(X2,Y2,test_size=0.7)

In [ ]:
from xgboost import XGBClassifier

liq_model = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
ob_fvg_model = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
liq_model.fit(train_x1, train_y1)
ob_fvg_model.fit(train_x2,train_y2)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# After training
y_pred1 = liq_model.predict(val_x1)
y_pred2 = ob_fvg_model.predict(val_x2)

print(classification_report(val_y1, y_pred1))
print(classification_report(val_y2, y_pred2))


In [ ]:
print(X2.columns)
print(X1.columns)

In [ ]:
labels = label_direction_based_on_zone_touch(df,fvg,ob,liq,threshold=0.1)

Data Combination of SMC Elements

In [ ]:
def smc_combine(df):
    merged = df.copy()
    merged = merged.reset_index(drop = True)
    merged['index'] = merged.index  # ensure index column exists
    fvg = detect_fvg(df)
    #fvg = check_fvg_reactions(df,fvg)
    ob = detect_order_blocks(df)
    #ob = check_ob_reactions(df,ob)
    swings = detect_swings(df,20)
    labeled_swings = label_structure_from_swings(swings)
    bos_choch=lookahead_bos_choch(labeled_swings,df,50)
    liq = detect_liquidity_zones(df,swings)
    #liq = check_liquidity_reactions(df,liq)

    fvg_df = pd.DataFrame(fvg).set_index('index')
    ob_df = pd.DataFrame(ob).set_index('index')
    bos_choch_df = pd.DataFrame(bos_choch).set_index('trigger_candle')
    labeled_swings_df = pd.DataFrame(labeled_swings).set_index('index')
    liq_df = pd.DataFrame(liq).set_index('start_index')
    merged = merged.merge(fvg_df, on='index', how='left', suffixes=('', '_fvg'))
    merged = merged.merge(ob_df, on='index', how='left', suffixes=('', '_ob'))
    merged = merged.merge(labeled_swings_df,on = 'index',how='left',suffixes=('','_swings'))
    merged = merged.merge(bos_choch_df, on='index', how='left', suffixes=('', '_bos_choch'))
    merged = merged.merge(liq_df, left_on='index', right_index=True, how='left', suffixes=('', '_liq'))
    merged['timestamp'] = df.index
    return merged

In [ ]:
smc_15m = smc_combine(df)
smc_1hr = smc_combine(df_1h)
smc_4hr = smc_combine(df_4h)
smc_1d = smc_combine(df_1d)

In [ ]:
def add_htf_features(base_df,htf_df,prefix):
    base_df = base_df.copy()
    htf_df = htf_df.copy()
    base_df = base_df.set_index('timestamp')
    htf_df = htf_df.set_index('timestamp')
    for feature in ['trend', 'fvg_high', 'fvg_low', 'ob_high', 'ob_low','level','swing_type']:
        if feature in htf_df.columns:
            base_df[f'{prefix}_{feature}'] = htf_df[feature].reindex(base_df.index, method='ffill')

    return base_df.reset_index()

In [ ]:
smc_15m = add_htf_features(smc_15m, smc_1hr, prefix='1h')
smc_15m = add_htf_features(smc_15m,smc_4hr, prefix='4h')
smc_15m = add_htf_features(smc_15m, smc_1d, prefix='1d')


In [ ]:
smc_15m.tail()

In [ ]:
smc_4hr.tail(20)

In [ ]:
smc_15m.to_csv('smc.csv',index=False)

In [ ]:
train_data = pd.read_csv('smc.csv')

# --- 1. Zone Confluence ---
train_data['fvg_ob_confluence'] = (
    (train_data['type'] == train_data['type_ob']) &
    train_data['type'].notna()
).astype(int)

train_data['fvg_liq_confluence'] = (
    (train_data['type'].notna()) &
    (train_data['type_liq'].notna()) &
    (abs(train_data['level'] - train_data['low']) < 1000)
).astype(int)

train_data['ob_liq_confluence'] = (
    (train_data['type_ob'].notna()) &
    (train_data['type_liq'].notna()) &
    (abs(train_data['level'] - train_data['low']) < 1000)
).astype(int)

# --- 2. Timing Features ---
train_data['time_since_bos_choch'] = train_data['index'] - train_data['trigger_candle']
train_data['delay_to_liquidity_sweep'] = train_data['swept_index'] - train_data['index']

# --- 3. Reaction Strengths ---
train_data['avg_max_price_move'] = train_data[[
    'max_price_move',
    'max_price_move_ob',
    'max_price_move_liq'
]].mean(axis=1)

train_data['has_strong_reaction'] = train_data['reaction_type'].isin(['clean bounce', 'bounce + break']).astype(int)

# --- 4. Labels ---
train_data['reaction_success'] = train_data['reaction_type'].isin(['clean bounce', 'bounce + break']).astype(int)
train_data['reaction_failed'] = (train_data['reaction_type'] == 'violation').astype(int)

train_data['setup_success'] = (
    (train_data['reaction_type_ob'].isin(['clean bounce'])) |
    (train_data['reaction_type'].isin(['clean bounce'])) |
    (train_data['reaction_type_liq'].isin(['clean bounce']))
).astype(int)

# Show the newly added features
train_data.tail()


In [ ]:
drop_cols = [
    'timestamp', 'index', 'price', 'trigger_candle', 'price_bos_choch',
    'trend_bos_choch', 'signal_bos_choch', 'trend_after_bos_choch','reaction_type',
    'reaction_type_ob',
    'reaction_type_liq','has_strong_reaction',
       'reaction_success', 'reaction_failed','price_retrace_ratio','max_price_move','price_retrace_ratio_ob',
       'max_price_move_ob','max_price_move_liq','avg_max_price_move','bounced','partial_mitigated','bounced_ob'
       ,'partial_mitigated_ob','bounced_liq' ,'violated','violated_ob','violated_liq'
]


In [ ]:
train_data.fillna({
    'structure_label': 'None',
    'signal': 'None',
    'trend': 'Unknown',

    'swing_type': 'None',
    'type': 'None',
    'type_ob': 'None',
    'type_liq': 'None'
}, inplace=True)

# Fill numeric NaNs
train_data.fillna(0, inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = [
    'structure_label', 'signal', 'trend',
    'swing_type', 'type', 'type_ob', 'type_liq'
]

le = LabelEncoder()
for col in categorical_cols:
    train_data[col] = le.fit_transform(train_data[col].astype(str))


In [ ]:
X = df_past_zones.drop(columns=['reaction_type','index','zone_high','zone_low'])
Y = df_past_zones['reaction_type']

In [ ]:
X = df_past_zones.drop(columns=['is_target','candle_index','zone_index','touched_index','zone_type'])
Y = df_past_zones['is_target']


In [ ]:
X = X.reset_index(drop= True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode all object columns
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# Cast entire DataFrame to float32
X = X.astype('float32')


In [ ]:
# Ensure all features are either int or float
cols = list(X.columns)
cols


In [ ]:
from sklearn.model_selection import train_test_split

train_x,val_x,train_y,val_y = train_test_split(X,Y,test_size=0.7)


In [ ]:
val_x.tail(5)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(train_x, train_y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=150,
    max_depth=6,
    random_state=42,
)

model.fit(train_x, train_y)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# After training
y_pred = model.predict(val_x)
y_proba = model.predict_proba(val_x)[:,1]

print(classification_report(val_y, y_pred))
print("AUC:", roc_auc_score(val_y, y_proba))


In [ ]:
# Get feature importance as dictionary
importance_dict = model.get_booster().get_score(importance_type='gain')  # You can also use: 'weight', 'cover', 'total_gain', 'total_cover'

# Convert to DataFrame for easier viewing
importance_df = pd.DataFrame.from_dict(importance_dict, orient='index', columns=['Importance'])
importance_df.index.name = 'Feature'
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

In [ ]:
reaction_type

In [ ]:
model.feature_importances_

In [ ]:
X.iloc[-40]

In [ ]:
model.predict(X.iloc[-41].values.reshape(1,-1))


In [ ]:
from joblib import dump, load


dump(model,'smcXGBoost.pkl')
print("Saved successfully")

In [ ]:
for k in train_data.iloc[50].keys():
    print(f'{k} : {train_data.iloc[50][k]}')